# Capstone Milestone 3: Federal Funds Rate Regression Modeling

In this project we will construct models for the Federal Funds Effective Rate, which we will just call the Federal Funds Rate (FFR). This is a rate determined by the market, similar to stock prices in the stock market. The FFR is a key tool used in monetary policy that influences economic activity. The Federal Reserve sets a Target for the FFR, then performs operations such as trading bonds to adjust the FFR, bringing it closer to the Target. Predicting the FFR benefits educators, economists, investors, financial institutions, and policy planners.

This project aims to first reproduce regression models predicting the FFR using Taylor’s Rule, a policy guideline by John Taylor from Stanford in 1993 and a modification of this used by researcher Alper D. Karakas, the equations of which are derived in Karkas’ (2023) paper, “Reevaluating the Taylor Rule with Machine Learning.”

We will then attempt to construct other models by adding the Target and Unemployment Rate to the Taylor Model to see if the addition of new features can improve the performance of regression models in predicting the Federal Funds Effective Rate. We choose to build off the Taylor Model as this is the foundational model and Karakas (2023) found little difference in performance between this model and their model.

We will also check the assumptions of regression for each model.

## Loading Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, linear_rainbow
from statsmodels.stats.stattools import durbin_watson, jarque_bera
from scipy.stats.mstats import winsorize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

## Defining Functions

In [ ]:
def fit_ols_model(X, y, model_name):
    """Fits an OLS Regression model for the given variables and returns predictions."""
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    # Get predictions
    y_pred = model.predict(X)
    return model, y_pred


def calculate_vif(X, model_name):
    """Calculates Variance Inflation Factors (VIFs)."""
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data


def error_metrics(y, y_pred):
    """Computes error metrics."""
    
    mse = round(mean_squared_error(y, y_pred), 3)
    rmse = round(np.sqrt(mse), 3)
    mae = round(mean_absolute_error(y, y_pred), 3)
    mpe = round(np.mean((y - y_pred) / y) * 100, 3)
    mape = round(np.mean(np.abs((y - y_pred) / y)) * 100, 3)
    r2 = r2_score(y, y_pred)
    return {
        "Mean Squared Error": mse,
        "Root Mean Squared Error": rmse,
        "Mean Absolute Error": mae,
        "Mean Percentage Error": mpe,
        "Mean Absolute Percentage Error": mape,
        "R-Squared": r2
    }


def fit_nn_model(X, y, model_name):
    """Fits a Neural Network model for the given variables and returns predictions and model."""
    # Set the input dimension
    input_dim = X.shape[1]

    # Define the neural network with an explicit Input layer
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(20, activation='relu'),
        Dense(1, activation='linear')
    ])

    # Compile the model and fit
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.fit(X, y, epochs=50, batch_size=5, verbose=0)

    # Get predictions
    y_pred = model.predict(X).flatten()
    return model, y_pred

## Data Wrangling

In [ ]:
# Define a dictionary with datasets and their URLs
datasets = {
    "ffer": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/FFER.csv",
    "pgdp": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/PGDP.csv",
    "rgdp": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/RGDP.csv",
    "cpi": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/CPI.csv",
    "fftr_lower": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/FFTR_lower.csv",
    "fftr_upper": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/FFTR_upper.csv",
    "fftr_old": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/FFTR_old.csv",
    "unrate": "https://raw.githubusercontent.com/nvpham12/Capstone-Project/refs/heads/main/UNRATE.csv"
}

# Iterate and load datasets
for name, url in datasets.items():
    globals()[name] = pd.read_csv(url, parse_dates=["observation_date"])
    print(f"{name} dataset loaded successfully.")

In [ ]:
# Iterate through the dataset names in the dictionary
for name in datasets.keys():
    # Access the DataFrame using globals()
    dataframe = globals()[name]
    duplicates = dataframe[dataframe.duplicated(keep=False)] 
    if not duplicates.empty:
        print(f"'{name}' has {len(duplicates)} duplicate rows:")
        print(duplicates)
    else:
        print(f"'{name}' has no duplicate rows.")

In [ ]:
# The data for CPI can be used to find inflation rates. 
# This is done to obtain a seasonally adjusted inflation dataset that isn't available on FRED.
inflation = pd.DataFrame()
inflation["observation_date"] = cpi["observation_date"] 
inflation["Inflation"] = cpi["CPIAUCSL"].pct_change(periods=12) * 100
inflation.dropna(inplace=True)

# Resample datasets to daily frequency.
inflation = inflation.set_index("observation_date").resample("D").ffill().reset_index()

pgdp = pgdp.set_index("observation_date").resample("D").ffill().reset_index()
rgdp = rgdp.set_index("observation_date").resample("D").ffill().reset_index()
unrate = unrate.set_index("observation_date").resample("D").ffill().reset_index()

# The Federal Funds Target Rate (FFTR) is set by the Federal Reserve. 
# The Fed used to set a single value as the target, but they shifted to setting a range.
# Find the midpoint of the range.

fftr_midpoint = pd.DataFrame()
fftr_midpoint["observation_date"] = fftr_upper["observation_date"] 
fftr_midpoint["Target"] = fftr_upper["DFEDTARU"] - fftr_lower["DFEDTARL"]

# Combine the midpoint with the old FFTR to get a complete FFTR dataset.
fftr_old = fftr_old.rename(columns = {"observation_date": "observation_date", "DFEDTAR": "Target"})
fftr = pd.concat([fftr_old, fftr_midpoint])

# Merge the dataframes
df = ffer.merge(inflation, on="observation_date", how="outer") \
        .merge(pgdp, on="observation_date", how="outer") \
        .merge(rgdp, on= "observation_date", how="outer") \
        .merge(unrate, on="observation_date", how="outer") \
        .merge(fftr, on="observation_date", how="outer")

# Set date as an index and rename the columns
df = df.set_index("observation_date")
df.columns = ["Federal Funds Rate", "Inflation (%)", "Potential GDP", "GDP", "Unemployment", "Target"]

In [ ]:
df

All datasets were obtained from the Federal Reserve Economic Database (FRED). Links to each dataset are in the reference list at the bottom of the Notebook.

We will need to derive some features:

Inflation gap: Inflation - Inflation target 

Output gap: Real GDP - Potential GDP

Inflation target is set at 2% by the Federal Reserve and is therefore treated as such our models (Karakas, 2023).

In [ ]:
# Find the inflation gap and output gap and add them to the dataframe
df["Inflation Gap"] = df["Inflation (%)"] - 2
df["Output Gap"] = df["GDP"] - df["Potential GDP"]

# Find Inflation Lag and Output Gap Lag for Karakas Model
df["Inflation Lag"] = df["Inflation (%)"].shift(1)
output_gap_lag = df["Output Gap"].shift(1)

# Create a percentage versions of Output Gap Lag and Inflation Lag for Karakas Model
df["Output Gap Lag %"] = (output_gap_lag / df["Potential GDP"]) * 100

# Drop rows with missing values from table
df.dropna(inplace=True)

# Drop Potential GDP and GDP columns as they are not needed
df = df.drop(["Potential GDP", "GDP"], axis=1)

In [ ]:
duplicates = df.index.duplicated(keep=False)
print(df[duplicates])

In [ ]:
print(df.isna().sum())

Karakas (2023) used vectors for the variables and changed the values to percentages. We need to create features to match this so that we can replicate the Karakas Model. We use first lagged variables as our vectors. Since the values of each of our variables can vary in numerical size, we apply Standard Scaling to the variables we will use for modeling to prevent the differences in size from affecting the models and their predictions.

We checked for duplicate rows earlier, but there were none. We checked again after merging the data sets. We dealt with missing values from the dataset by removing them completely.

In [ ]:
# Define the percentage of extreme values to cap
winsor_limits = (0.05, 0.05)

# Apply Winsorization to all numeric columns except the dependent variable
for col in df.columns:
    if col != "Federal Funds Rate":
        lower = np.percentile(df[col], winsor_limits[0] * 100)
        upper = np.percentile(df[col], 100 - winsor_limits[1] * 100)
        df[col] = np.clip(df[col], lower, upper)

Here, we capped extreme values using Winsorization. We set the percentage of extreme values to cap at 5%, which should remove most outliers with massive gaps from the rest of the data.

## Exploratory Data Analysis

In [ ]:
# Print the first 5 values of the dataframe
df.head()

In [ ]:
# Print the last 5 values of the dataframe
df.tail()

In [ ]:
# Print the shape of the dataframe, showing the number of rows (observations) and columns
print(df.shape)

In [ ]:
# Check the types of each variable
print(df.dtypes)

The data is entirely in float64, which is a data type that can be used for machine learning models in general.

In [ ]:
# Compute Summary Statistics
df.describe()

In [ ]:
for column in df.columns:
    plt.figure(figsize=(10, 5))
    sns.lineplot(df[column])
    plt.title(f'{column} over time')
    plt.show()

The FFR and Target have been steadily decreasing since the 1980s. Inflation, Output Gap, and Unemployment have roughly remained around a constant level over time, despite having some sharp rises or drops.

In [ ]:
# Construct a Correlation Heatmap
plt.figure(figsize=(10, 5))
sns.heatmap(
    df.corr(),
    annot=True,  # Display correlation values
    cmap="coolwarm",
    fmt=".2f",  # Limit to 2 decimal places
    linewidths=0.5,
)
plt.title("Correlation Heatmap", fontsize=16)
plt.xticks(rotation=45)  # Rotate x-axis labels for readability
plt.show()

Inflation, Inflation Gap, and Inflation Lag are perfectly correlated to each other, while Output Gap and Output Gap Lag (%) are very strongly correlated, which is expected. We will not be trying any models that use more than 1 in each set as predictors at a time.

Unemployment and Output Gap are strongly correlated, so we will need to watch out for these when checking the Variance Inflation Factors (VIFs) for multicollinearity issues.

## OLS Modeling

The variables used for each regression Model are listed as follows:

Taylor's Rule Model: 

    Dependent variable: Federal Funds Rate. 
    Independent variables: Output Gap, Inflation Gap

Karakas Model:

    Dependent variable: Federal Funds Rate. 
    Independent variables: Output Gap Lag %, Inflation Lag (%)
    
Target Model:

    Dependent variable: Federal Funds Rate. 
    Independent variables: Output Gap, Inflation Gap, Target
    
Unemployment Model:

    Dependent variable: Federal Funds Rate. 
    Independent variables: Output Gap, Inflation Gap, Unemployment
    
Both Model (uses both Unemployment and Target):

    Dependent variable: Federal Funds Rate. 
    Independent variables: Output Gap, Inflation Gap, Target, Unemployment

In [ ]:
# Apply Robust Scaler to independent variables
scaler = MinMaxScaler()
df_vars = ["Unemployment", "Target", "Inflation Gap", "Output Gap", "Output Gap Lag %", "Inflation Lag"]
df[df_vars] = scaler.fit_transform(df[df_vars])

In [ ]:
# Set dependent variable
y = df["Federal Funds Rate"]

# Define independent variables for each model
model_features = {
    "Taylor": df[["Output Gap", "Inflation Gap"]],
    "Karakas": df[["Output Gap Lag %", "Inflation Lag"]],
    "Target": df[["Output Gap", "Inflation Gap", "Target"]],
    "Unemployment": df[["Output Gap", "Inflation Gap", "Unemployment"]],
    "Both": df[["Output Gap", "Inflation Gap", "Target", "Unemployment"]],
}

# Initialize dictionaries to store results
ols_fitted_models = {}
ols_predictions = {}
ols_vif_results = {}
ols_error_metrics = {}

# Loop to fit, calculate VIFs, Calculate error metrics, and extract model stats for all models
for model_name, X in model_features.items():
    X = sm.add_constant(X)
    
    # Fit the model
    model, y_pred = fit_ols_model(X, y, model_name)
    ols_fitted_models[model_name] = model
    ols_predictions[model_name] = y_pred

    # Calculate VIF
    vif_data = calculate_vif(X, model_name)
    ols_vif_results[model_name] = vif_data
   
    # Calculate error metrics
    metrics = error_metrics(y, y_pred)
    ols_error_metrics[model_name] = metrics

### OLS Regression Assumptions

In [ ]:
# Dictionary to store assumption test results
ols_assumption_tests = {}
ols_residuals = {}

# Iterate through models
for model_name, model in ols_fitted_models.items():
    residuals = model.resid
    X = model.model.exog

    # Normality Tests (Jarque Bera)
    jb_test = jarque_bera(residuals)

    # Homoscedasticity Test (Breusch-Pagan)
    bp_test = het_breuschpagan(residuals, X)

    # Autocorrelation Test (Durbin-Watson)
    dw_stat = sm.stats.durbin_watson(residuals)

    # Linearity Tests (Rainbow)
    rainbow = linear_rainbow(model)

    # Store results
    ols_assumption_tests[model_name] = {
        "Durbin-Watson Test Statistic": f"{dw_stat:.6f}",
        "Jarque-Bera p-value": f"{jb_test[1]:.6f}",
        "Breusch-Pagan p-value": f"{bp_test[1]:.6f}",
        "Rainbow Test p-value": f"{rainbow[1]:.6f}"
    }
    
# Print results
for model, results in ols_assumption_tests.items():
    print(f"\nAssumption Test Statistics and P-values for {model}:")
    for test, value in results.items():
        print(f"{test}: {value}")

The assumptions of regression include:

1. Normality of residuals
2. Homoscedasticity
3. Autocorrelation
4. Linearity
5. Multicollinearity

These assumptions can be tested using the Jarque-Bera test (Normality), Breusch-Pagan test(Homoscedasticity), Durbin-Watson test (Autocorrelation), and Rainbow test (Linearity).
For every model, the above p-values are all around 0 and the Durbin Watson test statistic lies between 0 and 0.5. 

H0: The model does not violate the regression assumption

H1: The model does violate the regression assumption

Using the 95% confidence level (significance level 0.05), the Jarque-Bera test, Breusch-Pagan test, and Rainbow test statistics all have p-values of around 0, which is less than the significance level. Therefore, we would reject the null hypothesis, H0, that the models do not violate the corresponding regression assumptions. 

For the Durbin-Watson test, statistics less than 1 or greater than 3 indicate strong autocorrelation and would violate the regression assumption of autocorrelation.

The models violates the first 4 regression Assumptions, which makes its results completely unreliable. 

In [ ]:
# Print VIFs for all models
for model_name, vif_data in ols_vif_results.items():
    print(f"VIF for {model_name} Model:")
    print(vif_data)
    print("\n")

From the VIFs, we do not have serious problems with multicollinearity. There is moderate level of multicollinearity for Output Gap in the Both Model, but it is rather close to 5, which is the threshold for moderate levels and should not cause major issues if left alone. Unemployment, the other variable of interest from our correlation matrix analysis, has acceptable multicollinearity levels. Every other model has VIFs below 5, and mostly around 1. There are no serious issues of multicollinearity, which is the only regression assumption that can be considered passable for the models.

### Taylor and Karakas OLS Model Comparison

While all the regression assumptions except for Multicollinearity were violated and the results are unreliable, we can still compare our visualizations and metrics to Karakas's results and check if we reproduced their regression models, which is one objective of this project.

In [ ]:
print("Taylor Model Summary:\n")
print(ols_fitted_models["Taylor"].summary())

In [ ]:
print("Karakas Model Summary:\n")
print(ols_fitted_models["Karakas"].summary())

In [ ]:
# Taylor Predictions vs Actual Plot
selected_model = "Taylor"

plt.figure(figsize=(10, 5))
plt.plot(df.index, y, label="True Federal Funds Rate", color="royalblue")
plt.plot(df.index, ols_predictions[selected_model], label=f"{selected_model}'s Rule Predictions", color="red")
plt.title(f"{selected_model}'s Rule Predictions vs Federal Funds Rate")
plt.xlabel("Date")
plt.ylabel("Federal Funds Rate")
plt.legend()
plt.show()

In [ ]:
# Karakas Predictions vs Actual Plot
selected_model = "Karakas"

plt.figure(figsize=(10, 5))
plt.plot(df.index, y, label="True Federal Funds Rate", color="royalblue")
plt.plot(df.index, ols_predictions[selected_model], label=f"{selected_model}'s Rule Predictions", color="green")
plt.title(f"{selected_model}'s Rule Predictions vs Federal Funds Rate")
plt.title("Karakas vs Federal Funds Rate")
plt.xlabel("Date")
plt.ylabel("Federal Funds Rate")
plt.legend()
plt.show()

In [ ]:
# Taylor Predictions vs Karakas Predictions vs Actual Plot
plt.figure(figsize=(10, 5))
plt.plot(df.index, y, label="True Federal Funds Rate", color="royalblue")
plt.plot(df.index, ols_predictions["Taylor"], label="Taylor Model Predictions", color="red")
plt.plot(df.index, ols_predictions["Karakas"], label="Karakas Model Predictions", color="green")
plt.title("Taylor vs Karakas vs Federal Funds Rate")
plt.xlabel("Date")
plt.ylabel("Federal Funds Rate")
plt.legend()
plt.show()

The plots look similar to the ones shared in Karakas' (2023) paper. The date range is smaller in our plots, since the addition of other variables has restricted our date range due to missing values. However, for the dates that overlap between our plots and the ones Karakas shared in their paper, the plots do match. 

In [ ]:
# Assuming y contains actual values and y_pred contains predicted values
RSS_taylor = np.sum((y - ols_predictions["Taylor"]) ** 2)
print(f"Residual Sum of Squares (RSS):")
print(f"RSS for Taylor: {RSS_taylor:.2f}")
RSS_karakas = np.sum((y - ols_predictions["Karakas"]) ** 2)
print(f"RSS for Karakas: {RSS_karakas:.2f}")
print("\n")

# Assuming y contains actual values and y_pred contains predicted values
print(f"Sum of Absolute Errors (SAE):")
SAE_taylor = np.sum(np.abs(y - ols_predictions["Taylor"]))
print(f"SAE for Taylor: {SAE_taylor:.2f}")
SAE_karakas = np.sum(np.abs(y - ols_predictions["Karakas"]))
print(f"SAE for Karakas: {SAE_karakas:.2f}")

Like Karakas (2023), we found RSS and SAE values were higher for the Karakas Model. Having obtained similar results from the visualizations and metrics, our goal of reproducing the Taylor and Karakas Models has been successfully achieved. 

### OLS Model Metrics

In [ ]:
# Initialize a list to store model statistics
model_statistics = []

# Loop through the fitted models to extract key statistics
for model_name, model in ols_fitted_models.items():
    # Extract key values
    results = {
        "model": model_name,
        "adj_r_squared": round(model.rsquared_adj, 3),
        "aic": round(model.aic, 3),
        "bic": round(model.bic, 3),
        "f_stat": round(model.fvalue, 3),
        "f_p_value": round(model.f_pvalue, 3),
        "t_p_values": model.pvalues.round(3).tolist(),
    }
    # Append the results for each model
    model_statistics.append(results)

# Convert results to a DataFrame
statistics = pd.DataFrame(model_statistics)
statistics

Although the results may be unreliable, we find that the Taylor Model has higher values for R-squared, adjusted R-squared, but lower values for Akaike Information Criterion (AIC), and Bayesian Information Criterion (BIC) compared to the Karakas Model. The AIC and BIC are metrics that measure model quality, while considering complexity (number of features). Generally, the model with lower values for AIC and BIC is better than the model with higher values for them. This is consistent with how the Karakas Model explains less variance than the Taylor Model, as measured by adjusted R-squared.

While the results may not be reliable, notice that the inclusion of the Unemployment variable and the Target variable raises R-Squared very greatly. The Target Variable drives up R-Squared the most. Adding Unemployment to the model after including Target (resulting in the Both Model) raises R-Squared but only by a tiny amount. The inclusion of these two variables are still worth exploring in a different model.

In [ ]:
ols_error_metrics_df = pd.DataFrame(ols_error_metrics).T.round(4)
ols_error_metrics_df

The error metrics are also in favor of the Taylor Model having better performance over the Karakas Model. Karakas (2023) claimed that their model had more accurate predictions, but not by much, which we've found is not entirely correct. While the Karakas Model had better RSS and SAE values, which does indicate better fitness, other performance metrics show that, overall, the model does not perform as well as the Taylor Model, having larger average errors and percentage errors. The difference, however, is not very big.

## Neural Network Model

In [ ]:
# Initialize dictionaries to store results
nn_fitted_models = {}
nn_predictions = {}
nn_error_metrics = {}

# Loop to fit, calculate VIFs, Calculate error metrics, and extract model stats for all models
for model_name, X in model_features.items():
    model, y_pred = fit_nn_model(X, y, model_name)
    
    nn_fitted_models[model_name] = model
    nn_predictions[model_name] = y_pred
   
    # Calculate error metrics
    metrics = error_metrics(y, y_pred)
    nn_error_metrics[model_name] = metrics

### Neural Network Model Metrics

In [ ]:
nn_error_metrics_df = pd.DataFrame(nn_error_metrics).T.round(4)
nn_error_metrics_df

Our models have returned better results across the board for each model. The differences are most apparant for the Taylor and Karakas models which received a 0.2 increase in R-squared. While the increases in R-squared for the Target and Both models are less sizeable, they are rather huge considering the how high they were in the OLS model to begin with.

## Conclusion

Karakas (2023) noted that Taylor's Rule (and their variant) do not predict the FFR well and changed their model to neural networks, getting better predictions by capturing non-linear patterns. We note that while Karakas obtained poor results from both regression models they constructed, since all regression assumptions except for Multicollinearity have been violated, the results of those regressions are unreliable. We noticed that Karakas did not mention regression assumptions in their paper, so we decided to check them as part of the validation process. The results came as a surprise to us as we were not expecting regression assumptions to be violated. 

Through our findings from this project, we believe we can explain why both of Karakas' regression models had poor predictive performance. None of the independent variables, including but not limited to those used in Karakas' models, have a linear relationship with the FFR, which may be the reason why a non-linear model works better. 

Karakas (2023) also mentioned their model having predictions closer to the actual FFR, but not by much. However, the error metrics we computed show that it is the Taylor Model, rather than the Karakas Model that has smaller average errors and percentage errors.

We have tested (not shown in this notebook) other regression models (Generalized Least Squares and Huber regression) and transformations (Yeo-Johnson, Box-Cox, Log-transformation), but assumptions are still violated. Certain models and transformations may be able to satisfy linearity, but Normality, Homoscedastity, and Autocorrelation all remain problems. We believe that regression models are not well suited for predicting Taylor's Rule. We believe that non-linear models, such as neural networks, future research to predict the FFR should be focus on non-linear models. While the results may not be reliable due to violation of regression assumptions, it is worth noting that the inclusion of the Target and Unemployment variable did improve performance metrics.

After we created the neural network models, we found a similar pattern in model performance level: Karakas < Taylor < Unemployment < Target < Both. However, we now have more reliable metrics. While there was better performance from each model compared to the OLS model, we obtained significantly better performance with the inclusion of Unemployment and Target. However, the Target feature had a bigger effect on the model performance.

<h2><center>References</center></h2>

Board of Governors of the Federal Reserve System (US). (2025). Federal Funds Effective Rate [DFF]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/DFF

Board of Governors of the Federal Reserve System (US). (2025). Federal Funds Target Rate (DISCONTINUED) [DFEDTAR]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/DFEDTAR

Board of Governors of the Federal Reserve System (US). (2025). Federal Funds Target Range - Lower Limit [DFEDTARL]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/DFEDTARL

Board of Governors of the Federal Reserve System (US). (2025). Federal Funds Target Range - Upper Limit [DFEDTARU]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/DFEDTARU

Karakas, A. D. (2023). Reevaluating the Taylor Rule with Machine Learning. ArXiv.org. https://arxiv.org/abs/2302.08323

U.S. Bureau of Economic Analysis. (2025). Real Gross Domestic Product [GDPC1]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/GDPC1

U.S. Bureau of Labor Statistics. (2025). Consumer Price Index for all urban consumers: All items in U.S. city average [CPIAUCSL]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/CPIAUCSL

U.S. Bureau of Labor Statistics. (2025). Unemployment Rate [UNRATE]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/UNRATE

U.S. Congressional Budget Office. (2025). Real Potential Gross Domestic Product [GDPPOT]. Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/series/GDPPOT